In [1]:
"""
Created on Fri Jun 4 15:50 2020

This is a script to convert the NEMO temperature and salinity to potential temperature and practical salinity

@author: Clara Burgard
"""

'\nCreated on Fri Jun 4 15:50 2020\n\nThis is a script to convert the NEMO temperature and salinity to potential temperature and practical salinity\n\n@author: Clara Burgard\n'

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import gsw
import matplotlib.pyplot as plt
#import assess_param_funcs.useful_functions as uf
#import assess_param_funcs.T_S_profile_functions as tspf
#import assess_param_funcs.melt_functions as meltf
import multimelt.useful_functions as uf
import multimelt.T_S_profile_functions as tspf
import multimelt.melt_functions as meltf


import itertools

import distributed
import glob

In [3]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')


In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8795/status,
Dashboard: http://127.0.0.1:8795/status,Workers: 4
Total threads: 4,Total memory: 22.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40833,Workers: 4
Dashboard: http://127.0.0.1:8795/status,Total threads: 4
Started: Just now,Total memory: 22.35 GiB
Comm: tcp://127.0.0.1:36729,Total threads: 1
Dashboard: http://127.0.0.1:40613/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:39121,


In [5]:
%matplotlib inline

READ IN DATA

In [6]:
nemo_run = 'OPM018'

if nemo_run == 'OPM006':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM021':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM016' or nemo_run == 'OPM018':
    yy_start = 1980
    yy_end = 2008

In [7]:
inputpath_data='../../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_profiles='../../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_isf='../../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

outputpath_profiles='../../../data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [8]:
file_mask = xr.open_mfdataset(inputpath_data+'mask_variables_of_interest_Ant_stereo.nc', chunks={'x': 600, 'y': 600})
file_mask2 = xr.open_mfdataset(inputpath_data+'mask_depth_coord_Ant_stereo.nc')
file_isf = xr.open_mfdataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_new.nc', chunks={'x': 533, 'y': 533})
file_mask_cutted = uf.cut_domain_stereo(file_mask, map_lim, map_lim).squeeze().drop('time')
file_isf_cutted = uf.cut_domain_stereo(file_isf, map_lim, map_lim)#.squeeze().drop('time')

file_TS_orig  = xr.open_mfdataset(inputpath_data+'variables_of_interest_2000_Ant_stereo.nc', chunks={'x': 600, 'y': 600})
file_TS_orig_cutted = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim).squeeze().drop('time')

In [9]:
lon = file_isf_cutted['longitude']
lat = file_isf_cutted['latitude']

In [10]:
ts_files = list(sorted(glob.glob(inputpath_data+'variables_of_interest_*_Ant_stereo.nc')))
ds_ts  = xr.open_mfdataset(ts_files, concat_dim='new_time', combine='nested', chunks={'x': 600, 'y': 600})
ds_ts = ds_ts[['votemper', 'vosaline', 'sosst']]
ds_ts = ds_ts.squeeze('time')
ds_ts = ds_ts.rename({'new_time': 'time'})
ds_ts = ds_ts.assign_coords(time=np.arange(yy_start, yy_end+1))
ds_ts_cutted = uf.cut_domain_stereo(ds_ts, map_lim, map_lim)

Prepare the depth axis

In [11]:
nemo_depth = np.round(file_mask2['gdept_0'].squeeze(dim=['lon','lat']), 3) # round to mm scale - should be enough

Cut out the temperature and salinity and assign the new depth axis

In [12]:
ds_temp_saline_input = ds_ts_cutted[['votemper', 'vosaline']]
ds_temp_saline_input = ds_temp_saline_input.rename({'votemper': 'temperature', 'vosaline': 'salinity'})
ds_temp_saline_input = ds_temp_saline_input.rename({'deptht': 'depth'})
ds_temp_saline_input['depth'] = np.round(ds_temp_saline_input.depth, 3)
ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values)

In [13]:
ds_temp_saline_input['time']

<xarray.DataArray 'time' (time: 29)>
array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008])
Coordinates:
  * time     (time) int64 1980 1981 1982 1983 1984 ... 2004 2005 2006 2007 2008

In [14]:
#ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values).sel(time=range(1999,2011))

In [15]:
## only points where there is no bedrock - I think not needed for NEMO data
#vert_mask = file_in_T['z_bnds'][:,1] > interp_bed['bed'] 
# only points of open ocean
mask_ocean = np.isfinite(file_TS_orig_cutted['sosst']).squeeze()# == 1  #ocean without ice shelf cavity

CONVERT CONSERVATIVE TEMPERATURE FOR OPEN OCEAN REGIONS TO POTENTIAL TEMPERATURE 
AND ABSOLUTE SALIINITY TO PRACTICAL SALINITY

In [16]:
ds_temp_saline_input['theta_ocean'] = xr.apply_ufunc(gsw.pt_from_CT, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['temperature'].where(mask_ocean), dask = 'allowed')
ds_temp_saline_input['salinity_ocean'] = xr.apply_ufunc(gsw.SP_from_SA, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['depth'], lon, lat, dask = 'allowed')
ds_temp_saline_output = ds_temp_saline_input[['theta_ocean', 'salinity_ocean']]

Write the results to multiple files (1 per year)

In [17]:
yearly_datasets = list(tspf.split_by_chunks(ds_temp_saline_output.unify_chunks(),'time'))
paths = [tspf.create_filepath(ds, 'T_S_theta_ocean_corrected', outputpath_profiles, ds.time[0].values) for ds in yearly_datasets]

this takes approximately 1 min per year

In [18]:
xr.save_mfdataset(datasets=yearly_datasets, paths=paths)